In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/crime.csv')

In [ ]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'geometry', 'Community Area Name', 'F_Date', 'Time'],
      dtype='object')

In [ ]:
# 폭력 범죄 리스트 정의
violent_crimes = [
    "HOMICIDE", "CRIMINAL SEXUAL ASSAULT", "ROBBERY", "BATTERY", "RITUALISM",
    "ASSAULT", "BURGLARY", "THEFT", "MOTOR VEHICLE THEFT", "HUMAN TRAFFICKING", "ARSON"
]

# 폭력 범죄 여부 컬럼 생성
df['violent'] = df['Primary Type'].isin(violent_crimes).astype(int)

In [ ]:
def is_violent_crime(row):
    primary = row['Primary Type']
    description = str(row['Description']).upper()

    if primary in ['HOMICIDE', 'CRIMINAL SEXUAL ASSAULT', 'ROBBERY', 'HUMAN TRAFFICKING']:
        return True
    elif primary in ['ASSAULT', 'BATTERY', 'ARSON'] and 'AGGRAVATED' in description:
        return True
    else:
        return False

# 컬럼 생성
df['IsViolent'] = df.apply(is_violent_crime, axis=1).astype(int)

In [ ]:
# 날짜 데이터를 datetime 형식으로 변환
df['F_Date']  = pd.to_datetime(df['F_Date'])

# 분기(Quarter) 컬럼 생성
df['Quarter'] = df['F_Date'].dt.quarter

In [ ]:
high_risk_list = ['Humboldt Park',
                  'Austin',
                  'West Garfield Park',
                  'East Garfield Park',
                  'Near West Side',
                  'North Lawndale',
                  'Fuller Park',
                  'Grand Boulevard',
                  'Washington Park',
                  'Woodlawn',
                  'South Shore',
                  'Chatham',
                  'South Chicago',
                  'Roseland',
                  'West Pullman',
                  'Riverdale',
                  'New City',
                  'Chicago Lawn',
                  'West Englewood',
                  'Englewood',
                  'Greater Grand Crossing',
                  'Auburn Gresham']

df['high_risk'] = df['Community Area Name'].isin(high_risk_list).astype(int)

전체범죄

In [ ]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'geometry', 'Community Area Name', 'F_Date', 'Time',
       'violent', 'IsViolent', 'Quarter', 'high_risk'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# 1. 사용할 Feature 선택
features = [
    'Primary Type', 'Description', 'Community Area',
    'Latitude', 'Longitude', 'Quarter'
]
target = 'Arrest'

In [ ]:
#  범주형 컬럼 숫자로 변환 (Label Encoding)
label_cols = ['Primary Type', 'Description']
le_dict = {}

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # 문자열 변환 후 Label Encoding
    le_dict[col] = le

In [ ]:
# 학습 데이터 분리 (Train/Test)
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# 4. 모델 학습 (랜덤 포레스트)
model = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=-1)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# 예측
y_pred = model.predict(X_test)

In [ ]:
# 정확도 평가
accuracy = accuracy_score(y_test , y_pred)
print(f"Accuracy : {accuracy}")

Accuracy : 0.8702395587600094


In [ ]:
print(classification_report(y_test,y_pred,zero_division=0))

              precision    recall  f1-score   support

       False       0.89      0.94      0.92   1235014
        True       0.79      0.67      0.72    420321

    accuracy                           0.87   1655335
   macro avg       0.84      0.80      0.82   1655335
weighted avg       0.87      0.87      0.87   1655335



In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X, y, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

강력범죄

In [ ]:
# 강력범죄만 필터링
violent_df = df[df['violent']==1]

In [ ]:
violent_df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'geometry', 'Community Area Name', 'F_Date', 'Time',
       'violent', 'IsViolent', 'Quarter', 'high_risk'],
      dtype='object')

위험범죄

In [ ]:
# 위험범죄만 필터링
severe_df = df[df['IsViolent']]

In [ ]:
severe_df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'geometry', 'Community Area Name', 'F_Date', 'Time',
       'violent', 'IsViolent', 'Quarter'],
      dtype='object')